In [ ]:
import xlwt
import pandas as pd
import conllu

In [ ]:
test = conllu.parse(open("dante_01_test.conllu", "r").read())
udpipe2 = conllu.parse(open("udpipe2_test_clean.conllu", "r").read())

In [ ]:
ud_tags = ["ADJ", "ADV", "INTJ", "NOUN", "PROPN", "VERB", "ADP", "AUX", "CCONJ", "SCONJ", "DET", "NUM", "PART", "PRON", "PUNCT", "SYM", "X"]
wrong_tag_style = xlwt.easyxf(
        "font: name Arial, bold on, color white;"
        "pattern: pattern solid, pattern_fore_colour red, pattern_back_colour red;"
        "borders: left thin, right thin, top thin, bottom thin;"
    )

Clean tags

In [ ]:
def clean_tree(set_sents):
    for i, sent in enumerate(set_sents):
        for j, token in enumerate(sent):
            if token["upos"] not in ud_tags:
                # print("Updated: ", token["upos"])
                set_sents[i][j]["upos"] = "X" 
    return set_sents
udpipe2 = clean_tree(udpipe2)
test = clean_tree(test)

In [ ]:
sheets = {}

for ud_tag in ud_tags:
    sheets[ud_tag] = {}
    done = False
    for other_tag in ud_tags:
        sheets[ud_tag][other_tag] = {}

    for test_sent in test:
        for udpipe2_sent in udpipe2:
            if test_sent.metadata["sent_id"] == udpipe2_sent.metadata["sent_id"]:
                done = True
                for true_token, pred_token in zip(test_sent, udpipe2_sent):
                    if true_token["upos"] == ud_tag and true_token["upos"] != pred_token["upos"]:
                        sheets[ud_tag][pred_token["upos"]][ test_sent.metadata["sent_id"] ] = {"pred": [x['upos'] for x in udpipe2_sent], "true": [x['upos'] for x in test_sent], "words": [x['form'] for x in test_sent]}
                break


In [ ]:
for ud_tag, pred_tag in sheets.items():
    book = xlwt.Workbook()
    for sheet_type, sent_ids in pred_tag.items():
        if sheet_type == ud_tag:
            continue
        n_row = 0
        sheet = book.add_sheet(sheet_type, cell_overwrite_ok=True)
        row = sheet.row(n_row)
        row.write(0, f"Casos de {ud_tag} incorretamente etiquetados com {sheet_type}")
        n_row += 1
        for sent_id, tag_types in sent_ids.items():
            row = sheet.row(n_row)
            row.write(0, sent_id)
            for i, word in enumerate(tag_types["words"], start=1):
                row.write(i, word)
            
            n_row += 1
            row = sheet.row(n_row)
            row.write(0, "Referência")
            for i, tag in enumerate(tag_types["true"], start=1):
                row.write(i, tag)
            
            n_row += 1
            row = sheet.row(n_row)
            row.write(0, "Predição")
            for i, tag in enumerate(tag_types["pred"], start=1):
                # DEBUG
                # if tag == "ADJ" and sheet_type == "ADV" and tag_types["true"][i - 1] == "ADJ":
                #     print(tag_types["true"][i - 1], tag_types["pred"][i - 1])
                if tag_types["true"][i - 1] != tag and \
                   tag_types["true"][i - 1] == ud_tag and \
                   tag_types["pred"][i - 1] == sheet_type:
                    row.write(i, tag, wrong_tag_style)
                else:
                    row.write(i, tag)
                    
            n_row += 2
        book.save(f"{ud_tag}.xlsx")